## Colormaps for Categorical Data

Colorcet originally included only continuous color spaces, but it was created while working on [Datashader](http://datashader.org), which uses both continuous and categorical sets of colors.  For rendering categorical data, Datashader requires having a distinct color per category, and it includes some ad-hoc collections of colors to try to get enough distinct colors to deal with complex datasets.  

A principled approach for designing arbitrarily large sets of mutually distinct colors was presented in:

> [Glasbey, Chris; van der Heijden, Gerie & Toh, Vivian F. K. et al. (2007)](https://strathprints.strath.ac.uk/30312/1/colorpaper_2006.pdf), "Colour displays for categorical images", Color Research & Application 32.4: 304-309.

"Glasbey" colors are available in [ImageJ](http://imagej.net/Glasbey) and for [R](https://github.com/btupper/catecolors), and there is a [Python implementation](https://github.com/taketwo/glasbey) of the method.  Generating the colors with the Python code is time consuming, so we generated some large sets of Glasbey colors and distribute them in colorcet.

In [ ]:
import numpy as np
import colorcet as cc
import holoviews as hv

hv.extension('bokeh')

We'll use bokeh for displaying these colormaps so that you can zoom in for a closer look.

In [ ]:
xs, _ = np.meshgrid(np.linspace(0, 1, 256), np.linspace(0, 1, 10))

def colormap(name, cmap=None, bounds=(0, 0, 256, 1), array=xs, **kwargs):
    if cmap is None:
        cmap = cc.palette[name]
    options = hv.opts.Image(xaxis=None, yaxis=None, width=900, height=100, cmap=cmap, toolbar='above', **kwargs)
    return hv.Image(array, bounds=bounds, group=name).opts(options)

The code requires some starting set of colors, which by default is just white. Using the default starting set, the second color is always very dark, which doesn't make for a very recognizable first color to choose for plotting. By adding both white and black into the starting colors, the initial colors in the subsequent sequence seem more useful for plotting, in that they are actually nameable, saturated colors. All glasbey colormaps included in colorcet are initialized with both black and white. In addition, some add the named categorical colormaps such as Category10 and Category20 used in [bokeh](https://bokeh.pydata.org/en/latest/docs/reference/palettes.html#bokeh.palettes.d3) and originally from [D3](https://github.com/d3/d3-3.x-api-reference/blob/master/Ordinal-Scales.md#categorical-colors) into the starting set, to allow them to be drop-in replacements and extensions for those common categorical colormaps. The naming of these colormaps follows a different pattern than that of continuous colormaps:
       
       glasbey_<starting_palette>[_<no_gray|dark|light>]

**NOTE:** Although the colormaps are initialized with black and white, black and white are not included in the final colormap. All colormaps are accessible as lists, so if you would like to include black and white - just add them to the top of the list.

In [ ]:
hv.Layout([colormap(name) for name in sorted(list(cc.palette.keys())) if "glasbey" in name]).cols(1)

In addition to these long form names, colorcet provides several aliases to the most useful of the glasbey colormaps. These include - `glasbey`, `glasbey_cat10`, and `glasbey_cat20`. `glasbey_cat10` and `glasbey_cat20` can be used as replacements for the bokeh `Category10` and `Category20` colormaps since they start with the same sequence.

In [ ]:
hv.Layout([
    colormap('glasbey'), 
    colormap('glasbey_cat10'), 
    colormap('glasbey_cat20')
]).cols(1)

To check how these compare with the original sets in bokeh, we can render them side by side. Drag to the right or scroll out to see more of the glasbey versions.

In [ ]:
from bokeh.palettes import Category10_10, Category20_20

hv.Layout([
    colormap('bokeh: Category10', cmap=Category10_10, bounds=(0, 0, 10, 1)), 
    colormap('glasbey_cat10')
]).cols(1).redim.range(x=(0, 50))

In [ ]:
hv.Layout([
    colormap('bokeh: Category20', cmap=Category20_20, bounds=(0, 0, 20, 1)), 
    colormap('glasbey_cat20')
]).cols(1).redim.range(x=(0, 50))